In [ ]:
#| notest
#| eval: false
#| hide

import sys

sys.path.insert(0, '../unreal_llm_sandbox/')
sys.path.insert(0, '../../../')

import big_project_helper as bph

bph.display_project_contents('unreal_llm_sandbox')

In [ ]:
CELL_SCRIPT = """
console.log('CODE_CELL_SCRIPT loaded successfully!');

window.cellOutputs = window.cellOutputs || {};


/**
 * Toggles the 'minimized' class on a cell element to collapse or expand it.
 * 
 * @param {string} cellId - The unique identifier of the cell to toggle.
 */
 function toggleMinimize(cellId){
    const cell = document.querySelector(`[data-cell-id="${cellId}"]`);
    cell.classList.toggle('minimized');
}


/**
 * Retrieves the code content from a Monaco editor instance within a cell.
 * 
 * @param {string} cellId - The unique identifier of the cell containing the Monaco editor.
 * @returns {string} The current text content of the Monaco editor model.
 */
 function getMonacoContent(cellId){
    const cell = document.querySelector(`[data-cell-id="${cellId}"]`);
    const monaco_container = cell.querySelector('.monaco-editor');
    const monaco_editor = monaco_container.querySelector('.monaco-editor');
    const monaco_model = monaco.editor.getModels().find(
        m => m.uri.toString() === monaco_editor.dataset.uri
    );
    const monaco_code = monaco_model.getValue();
    return monaco_code;
}


/**
 * Clears both the output store and output display elements within a cell.
 * 
 * @param {string} cellId - The unique identifier of the cell to clear.
 */
function clearOutput(cellId) {
    const cell = document.querySelector(`[data-cell-id="${cellId}"]`);
    const cell_type = cell.getAttribute('data-cell-type');

    // Clear Output Store
    // For code cells, initialize with empty array
    if (cell_type == 'code' || cell_type == 'prompt')
    {
        const output_store = cell.querySelector('.output-store');    
        if (cell_type === 'code') {
            output_store.textContent = "[]";
        }
        else if (cell_type === 'prompt') {
            output_store.textContent = "";
        }

        // Clear Output Display
        const output_display = cell.querySelector('.output-display');
        output_display.textContent = "";
    }

    else if (cell_type === 'agent') {
        const output_store_llm = cell.querySelector('.output-store-llm');    
        const output_store_code = cell.querySelector('.output-store-code');    

        output_store_llm.textContent = "";
        output_store_code.textContent = "[]";

        const output_display_llm = cell.querySelector('.output-display-llm');
        output_display_llm.textContent = "";
        const output_display_code = cell.querySelector('.output-display-code');
        output_display_code.textContent = "";

    }
}

/**
 * Executes a prompt cell by sending its content to the LLM endpoint and streaming the response.
 * Clears previous output, gathers conversation context from unreal_llm_sandbox.cells above, and renders
 * the streamed response into the cell's output area.
 * 
 * @param {string} cellId - The unique identifier of the prompt cell to execute.
 */
function executePromptCell(cellId)
{
    // Clear Cell
    clearOutput(cellId);

    // Get Output Area to Write To
    const cell = document.querySelector(`[data-cell-id="${cellId}"]`);
    const output_store = cell.querySelector('[class="output-store"]');

    // Turn Off Edit Mode
    const toggle_edit = cell.querySelector('.toggle-edit');
    toggle_edit.checked = false;

    // Prep Data
    const chat_history = gatherCellsForLLM(cellId);

    const prompt = extractCellInput(cellId);
    
    const notebook_name = document.querySelector('.notebook-name')?.value || 'untitled';

    const chat_data = {
        'prompt': prompt,
        'context': chat_history,
        'notebook': notebook_name 
    };

    (async () => {
    const { fetchEventSource } = await import('https://esm.sh/@microsoft/fetch-event-source@2.0.1');
    await fetchEventSource(`/execute_prompt/${cellId}`, {
        method: 'POST',
        headers: {'Content-Type': 'application/json'},
        body: JSON.stringify(chat_data),
        onmessage(ev) {
            try {
                const packet = JSON.parse(ev.data);
                if (packet.type === 'text') {
                    output_store.textContent += packet.data;
                }
            } catch(error) {
                output_store.textContent += `\n ** Error In Stream ** \n`;
            }
        },
        onerror(error) {
            output_store.textContent += `\n ** Connection of Server Error ** \n` ;
            console.error(error);
        },
    });
    })();
}


/**
 * Extracts the input content from a cell based on its type.
 * For code cells, retrieves content from the Monaco editor.
 * For other cell types (note/prompt), retrieves content from the textarea.
 * 
 * @param {string} cellId - The unique identifier of the cell to extract input from.
 * @returns {string} The text content of the cell's input area.
 */
function extractCellInput(cellId)
{
    const cell = document.querySelector(`[data-cell-id="${cellId}"]`);
    const cell_type = cell.getAttribute('data-cell-type');
    
    if (cell_type == 'code')
        {
            return getMonacoContent(cell.getAttribute('data-cell-id'));
        }
    else
        {
            const textarea = cell.querySelector("textarea");
            return textarea.value;
        }
}


/**
 * Executes a code cell by sending its content to the backend for execution via SSE stream.
 * Clears previous output, retrieves code from the Monaco editor, and streams
 * execution results back to the cell's output area.
 * 
 * @param {string} cellId - The unique identifier of the code cell to execute.
 */
function executeCell(cellId)
{
    // Clear Output
    clearOutput(cellId);

    /// Prep Monaco
    const code = getMonacoContent(cellId);

    const notebook_name = document.querySelector('.notebook-name')?.value || 'untitled';

    const code_data = {
        'code':code,
        'notebook': notebook_name 
    };

    // Send Post and Start Stream
    (async () => {
    const { fetchEventSource } = await import('https://esm.sh/@microsoft/fetch-event-source@2.0.1');
    await fetchEventSource(`/execute_code/${cellId}`, {
        method: 'POST',
        headers: {'Content-Type': 'application/json'},
        body: JSON.stringify(code_data),
        onmessage(ev) {
            const packet = JSON.parse(ev.data);
            if (packet.type === 'output') {
                accumulateOutput(cellId, packet.data);  // packet.data contains kernel msg
            }
        },
        onerror(err) {
            console.error('Stream error:', err);
            throw err; // stops retry
        }
    });
    })();
}


/**
 * Accumulates streamed execution output into a cell's output store.
 * Handles different Jupyter message types (stream, execute_result, display_data, error)
 * and merges consecutive stream outputs of the same name (stdout/stderr) into a single entry.
 * 
 * @param {string} cellId - The unique identifier of the cell to accumulate output for.
 * @param {Object} data_dict - The incoming message from the execution stream.
 * @param {string} data_dict.msg_type - Type of message: 'stream', 'execute_result', 'display_data', or 'error'.
 * @param {Object} data_dict.content - The message payload, structure varies by msg_type.
 */
function accumulateOutput(cellId,data_dict)
{
        //data_dict = JSON.parse(ev.data);

        // Get cell and output
        const cell = document.querySelector(`[data-cell-id="${cellId}"]`);
        const output_store = cell.querySelector('.output-store');

        let accumulated_data = [];
        // Unpack content
        if (output_store.textContent)
        {
            accumulated_data = JSON.parse(output_store.textContent);
        }

        let acc_len = accumulated_data.length;

        let prev_msg_type = null;
        // store previous message type
        if (acc_len > 0)
        {
            prev_msg_type = accumulated_data[acc_len-1]['output_type'];
        }

        if (data_dict['msg_type'] == 'stream')
        {
            let added = false;
            if (prev_msg_type == 'stream')
            {
                // Append to the same list if text stream
                if (accumulated_data[acc_len-1]['name'] == data_dict['content']['name'] )
                {
                    accumulated_data[acc_len-1]['text'].push(data_dict['content']['text']);
                    added = true;
                }

            }
            if (!added)
            {
            accumulated_data.push( {'output_type':'stream',
                                'name':data_dict['content']['name'],
                                'text':[data_dict['content']['text']]});

            }

        }
        else if (data_dict['msg_type'] == 'execute_result')
        {
            accumulated_data.push( {'output_type':'execute_result',
                                'data':data_dict['content']['data'],
                                'execution_count':data_dict['content']['execution_count']});
        }
        else if (data_dict['msg_type'] == 'display_data')
        {
            accumulated_data.push( {'output_type':'display_data',
                                'data':data_dict['content']['data']});
        }
        else if (data_dict['msg_type'] == 'error')
        {
            accumulated_data.push( {'output_type':'error',
                                'ename':data_dict['content']['ename'],
                                'evalue':data_dict['content']['evalue'],
                                'traceback':data_dict['content']['traceback']});
        }
        output_store.textContent = JSON.stringify(accumulated_data);
}


/**
 * Renders the output of a prompt cell by parsing its stored markdown content
 * and displaying the formatted HTML with syntax highlighting.
 * Retrieves raw markdown from the output store, converts it to HTML using marked.js,
 * and applies Prism.js syntax highlighting to any code blocks.
 * 
 * @param {string} cellId - The unique identifier of the prompt cell to render.
 */
function renderOutputPrompt(cellId,tag='')
{
    cell = document.querySelector(`[data-cell-id="${cellId}"]`);
    output_store = cell.querySelector('.output-store'+tag);
    output_display = cell.querySelector('.output-display'+tag);
    markdown = marked.parse(output_store.textContent);
    output_display.innerHTML = ''
    if (markdown)
    {
        output_display.innerHTML = markdown;
        Prism.highlightAllUnder(output_display);
    }
}


/**
 * Converts a Jupyter-style MIME bundle into renderable HTML.
 * Checks for image formats (PNG, JPEG), HTML, and plain text in priority order,
 * returning the appropriate HTML string for display.
 * 
 * @param {Object} data - A MIME bundle object with keys like 'image/png', 'text/html', etc.
 * @returns {string|undefined} HTML string for the highest-priority format found, or undefined if none match.
 */
function renderData(data)
{
   if (data.hasOwnProperty("image/png")){
      return `<img src="data:image/png;base64,${data['image/png']}">`;
   }
   else if (data.hasOwnProperty("image/jpeg")){
      return `<img src="data:image/jpeg;base64,${data['image/jpeg']}">`;
   }
   else if (data.hasOwnProperty("text/html")){
      return data['text/html'];
   }
   else if (data.hasOwnProperty("text/plain")){
      return `<div class="result-output">${data['text/plain']}</div>`;
   }
}


/**
 * Renders the accumulated execution output for a code cell by parsing the stored JSON
 * and converting it to displayable HTML. Handles multiple Jupyter output types:
 * - stream: Concatenates text chunks into a single div (stdout/stderr)
 * - execute_result/display_data: Delegates to renderData() for MIME bundle rendering
 * - error: Formats exception name, value, and ANSI-colored traceback
 * 
 * Uses AnsiUp to convert ANSI escape codes in tracebacks to styled HTML.
 * 
 * @param {string} cellId - The unique identifier of the code cell to render output for.
 */
function renderOutput(cellId, tag='')
{
    ansi_up = new AnsiUp();
    const cell = document.querySelector(`[data-cell-id="${cellId}"]`);
    const output_store = cell.querySelector('.output-store'+tag);
    const output_display = cell.querySelector('.output-display'+tag);

    const data = JSON.parse(output_store.textContent)

    let out_html = ''

    for (i = 0; i < data.length; i++)
        {
            if (data[i]['output_type'] == 'stream')
            {
                const text_list = data[i]['text'];
                const text_len = text_list.length;

                out_html += '<div class="stream-output">';
                for (s = 0; s < text_len; s++  )
                    {
                         out_html += text_list[s];
                    }
                out_html += `</div>`;
            }
            else if (data[i]['output_type'] == 'execute_result' || data[i]['output_type'] == 'display_data' )
            {
                out_html += renderData(data[i]['data']);
            }
            else if (data[i]['output_type'] == 'error')
            {
                const trace_list = data[i]['traceback'];
                const trace_len = trace_list.length;

                // out_html += `<div class="error-name">${data[i].ename}: ${data[i].evalue}</div>`;
                out_html += '<pre class="traceback">';
                for (s = 0; s < trace_len; s++  )
                    {
                         out_html += ansi_up.ansi_to_html(trace_list[s]);
                         out_html += `\n`;
                    }
                out_html += '</pre>';

            }

        }
    output_display.innerHTML = out_html; 
}


/**
 * Removes a cell element from the DOM by its unique identifier.
 * 
 * @param {string} cellId - The unique identifier of the cell to delete.
 */
 function deleteCell(cellId) {
    document.querySelector(`[data-cell-id="${cellId}"]`).remove();
}


/**
 * Moves a cell down by swapping its position with its next sibling element.
 * If no next sibling exists, the cell remains in place.
 * 
 * @param {string} cellId - The unique identifier of the cell to move down.
 */
 function moveDown(cellId){
    const cell = document.querySelector(`[data-cell-id="${cellId}"]`);
    // Check Sibling
    const sibling = cell.nextElementSibling;
    if (sibling){
        cell.parentNode.insertBefore(sibling,cell);
    }
}


/**
 * Moves a cell up by swapping its position with its previous sibling element.
 * If no previous sibling exists, the cell remains in place.
 * 
 * @param {string} cellId - The unique identifier of the cell to move up.
 */
function moveUp(cellId){
    const cell = document.querySelector(`[data-cell-id="${cellId}"]`);
    
    // Check Sibling
    const sibling = cell.previousElementSibling;
    if (sibling){
        cell.parentNode.insertBefore(cell,sibling);
    }
}


/**
 * Gathers all cells above a given cell and formats them for LLM context.
 * Collects each cell's id, type, source content, and outputs until reaching the target cell.
 * 
 * @param {string} cellId - The unique identifier of the cell to stop at (exclusive).
 * @returns {Array<Object>} Array of cell objects with cell_id, cell_type, source, and outputs.
 */
function gatherCellsForLLM(cellId){
    let cell_out_list = [];
    const cells = document.querySelectorAll('[data-cell-id]');
    for (let i = 0; i < cells.length; i++){

        const cell = cells[i];
        const id = cell.getAttribute('data-cell-id');
        const type = cell.getAttribute('data-cell-type');
        const input = extractCellInput(id);
        const output = extractCellOutput(id);

        if (cellId == id)
        {
            break;
        }
    
        const cell_dict = {'cell_id':id,
                    'cell_type':type,
                    'source':input,
                    'outputs':output,
                    };
        cell_out_list.push(cell_dict);
    }
    return cell_out_list;
}


/**
 * Extracts the output content from a cell based on its type.
 * For code cells, parses JSON from the output store.
 * For prompt cells, returns the raw text content.
 * 
 * @param {string} cellId - The unique identifier of the cell.
 * @returns {Array|string|null} Parsed output array for code cells, string for prompt cells, null otherwise.
 */
function extractCellOutput(cellId){
    const cell = document.querySelector(`[data-cell-id="${cellId}"]`);
    const cell_type = cell.getAttribute('data-cell-type');
    const output_store = cell.querySelector('.output-store');
    
    if (cell_type == 'code')
        {
            try{
                return JSON.parse(output_store.textContent);
            }catch{
                return [];
            }
        }
    else if (cell_type == 'prompt')
        {
            return output_store.textContent;
        }
    else if (cell_type == 'agent')
    {
        return getMonacoContent(cellId);
    }
        
    else{
        return null;
    }
}


/**
 * Sets up a MutationObserver to watch a cell's output store for changes
 * and automatically re-renders the output when changes are detected.
 * Handles both code cells (renderOutput) and prompt cells (renderPromptOutput).
 * 
 * @param {string} cellId - The unique identifier of the cell to watch.
 */
function watchOutputStore(cellId) {
    const cell = document.querySelector(`[data-cell-id="${cellId}"]`);
    const cell_type = cell.getAttribute('data-cell-type');

    // Throttle render to ~15fps
    let renderPending = false;
    function throttledRender(fn) {
        if (renderPending) return;
        renderPending = true;
        setTimeout(() => {
            fn();
            renderPending = false;
        }, 30);
    }

    const config = { characterData: true, childList: true, subtree: true };

    // Initial render
    if (cell_type == 'code') {
        renderOutput(cellId);
    } else if (cell_type == 'prompt') {
        renderOutputPrompt(cellId);
    } else if (cell_type == 'agent') {
        renderOutputPrompt(cellId, '-llm');
        renderOutput(cellId, '-code');
    }

    // Set up observers
    if (cell_type == 'code') {
        const output_store = cell.querySelector('.output-store');
        new MutationObserver(() => throttledRender(() => renderOutput(cellId)))
            .observe(output_store, config);
    } else if (cell_type == 'prompt') {
        const output_store = cell.querySelector('.output-store');
        new MutationObserver(() => throttledRender(() => renderOutputPrompt(cellId)))
            .observe(output_store, config);
    } else if (cell_type == 'agent') {
        const output_store_llm = cell.querySelector('.output-store-llm');
        const output_store_code = cell.querySelector('.output-store-code');
        new MutationObserver(() => throttledRender(() => renderOutput(cellId, '-code')))
            .observe(output_store_code, config);
        new MutationObserver(() => throttledRender(() => renderOutputPrompt(cellId, '-llm')))
            .observe(output_store_llm, config);
    }
}

async function saveNotebook(filename) {
    const cells = gatherCellsForLLM(null);  // null = gather all cells
    await fetch(`/save_notebook/${filename}`, {
        method: 'POST',
        headers: {'Content-Type': 'application/json'},
        body: JSON.stringify({cells: cells})
    });
    console.log('Notebook saved!');
}

setInterval(() => {
    const notebookName = document.querySelector('.notebook-name')?.value || 'untitled';
    saveNotebook(notebookName+'.ipynb');
}, 2000);


"""

In [ ]:

HEIGHT_ADJ_SCRIPT="""
document.addEventListener('DOMContentLoaded', () => {
    // Adjust all textareas on load + input
    document.querySelectorAll('textarea[class*="content-edit"]').forEach(textarea => {
        textarea.style.height = 'auto';
        textarea.style.height = Math.min(textarea.scrollHeight, 500) + 'px';
        
        textarea.addEventListener('input', () => {
            textarea.style.height = 'auto';
            textarea.style.height = Math.min(textarea.scrollHeight, 500) + 'px';
        });
    });

    // Adjust when toggling to edit mode
    document.querySelectorAll('.toggle-edit').forEach(checkbox => {  // Changed selector
        checkbox.addEventListener('change', () => {
            if (checkbox.checked) {
                const cell = checkbox.closest('[data-cell-id]');  // Changed - find parent cell
                const textarea = cell?.querySelector('textarea.content-edit');  // Changed
                if (textarea) {
                    textarea.style.height = 'auto';
                    textarea.style.height = Math.min(textarea.scrollHeight, 500) + 'px';
                }
            }
        });
    });
});
"""

CELL_FOCUS_SCRIPT = """
let currentFocusedCellId = null;

document.addEventListener('click', (e) => {
    const cell = e.target.closest('[data-cell-id]');  // Changed from [type="cell"]
    
    // Remove outline from all cells
    document.querySelectorAll('[data-cell-id]').forEach(c => {  // Changed
        c.style.outline = 'none';
    });
    
    // Add outline to clicked cell
    if (cell) {
        cell.style.outline = '2px solid #3b82f6';
        cell.style.outlineOffset = '-8px';
        cell.style.borderRadius = '16px';
        currentFocusedCellId = cell.dataset.cellId;  // Changed from cell.id
    }
    document.querySelectorAll('.toggle-edit').forEach(t => {  // Changed selector
        const tCell = t.closest('[data-cell-id]');
        if (tCell && tCell.dataset.cellId !== cell?.dataset.cellId) {
            t.checked = false;
        }
    });
    const isContentArea = e.target.closest('.markdown-body') || 
                        e.target.closest('textarea[class*="content-edit"]');

    if (isContentArea && cell) {
        const toggleCheckbox = cell.querySelector('.toggle-edit');  // Changed - find within cell
        if (toggleCheckbox) {
            toggleCheckbox.checked = true;
            toggleCheckbox.dispatchEvent(new Event('change'));
        }
    }
});
"""


In [ ]:

AGENT_SCRIPT ="""

/**
 * Retrieves the code content from a Monaco editor instance within a cell.
 * 
 * @param {string} cellId - The unique identifier of the cell containing the Monaco editor.
 * @returns {string} The current text content of the Monaco editor model.
 */
function setMonacoContent(cellId, txt) {
    const cell = document.querySelector(`[data-cell-id="${cellId}"]`);
    const monaco_container = cell.querySelector('.monaco-editor');

    // 1. Get the URI (ensure you grab it from the correct element in your DOM)
    const uri = monaco_container.dataset.uri || 
                monaco_container.querySelector('[data-uri]')?.dataset.uri;

    const monaco_model = monaco.editor.getModels().find(
        m => m.uri.toString() === uri
    );

    if (monaco_model) {
        monaco_model.setValue(txt);

        // 2. Find the editor instance attached to this model
        const editorInstance = monaco.editor.getEditors().find(
            e => e.getModel() === monaco_model
        );

        // 3. Use the REAL method to scroll to the last line
        if (editorInstance) {
            editorInstance.revealLine(monaco_model.getLineCount());
        }
    }
}


/**
 * Executes a prompt cell by sending its content to the LLM endpoint and streaming the response.
 * Clears previous output, gathers conversation context from unreal_llm_sandbox.cells above, and renders
 * the streamed response into the cell's output area.
 * 
 * @param {string} cellId - The unique identifier of the prompt cell to execute.
 */
function executeAgentCell(cellId)
{
    // Clear Cell
    clearOutput(cellId);


    const error_text = `\n ** Error In Stream ** \n` ;
    // Get Output Area to Write To
    const cell = document.querySelector(`[data-cell-id="${cellId}"]`);
    const output_store_llm = cell.querySelector('[class="output-store-llm"]');
    const output_store_code = cell.querySelector('[class="output-store-code"]');
    const status_div = cell.querySelector('#status');
    const iteration_div = cell.querySelector('#iteration');
    const output_display_llm = cell.querySelector('.output-display-llm');
    const output_display_code = cell.querySelector('.output-display-code');


    // Turn Off Edit Mode
    const toggle_edit = cell.querySelector('.toggle-edit');
    toggle_edit.checked = false;

    // Prep Data

    const prompt = extractCellInput(cellId);
    const existingCode = getMonacoContent(cellId);
    const hasCode = existingCode && existingCode.trim().length > 0;

    const chat_history = gatherCellsForLLM(cellId);

    const notebook_name = document.querySelector('.notebook-name')?.value || 'untitled';

    const chat_data = {
        'prompt': prompt,
        'existing_code': hasCode ? existingCode : null,
        'context': chat_history  ,
        'notebook': notebook_name 
    };
    
    setMonacoContent(cellId,'')

    let tag;
    let iter = 0;
    (async () => {
    const { fetchEventSource } = await import('https://esm.sh/@microsoft/fetch-event-source@2.0.1');
    await fetchEventSource(`/agent_tool_build/${cellId}`, {
        method: 'POST',
        headers: {'Content-Type': 'application/json'},
        body: JSON.stringify(chat_data),
        onclose() {
        throw new Error('Stream closed');  // Prevents retry
        },
        onmessage(ev) {


            // Try to recieve packet
            try{

                const packet = JSON.parse(ev.data)
                const { type, data } = packet;

                //////////////////////
                // Set GUI Tags / Labels
                //////////////////////
                if (type == 'tag')
                {
                    if ( data.includes('review-box'))
                    {
                        status_div.textContent = 'Reviewing Unit Test';
                        tag = 'llm';
                        output_store_llm.textContent = ''
                    }
                    if (data.includes('code-box' ))
                    {
                        iter += 1
                        iteration_div.textContent = 'Iteration: '+iter.toString();
                        
                        status_div.classList.remove('complete');
                        status_div.classList.add('running');
                        status_div.textContent = 'Writing Code';

                        // When finished successfully

                        tag = 'code';
                        setMonacoContent(cellId,'')

                    }
                    if (data.includes('unit-box' ))
                    {
                        status_div.textContent = 'Exectuting Unit Test';
                        tag = 'unit';
                  
                    }
                    if (data.includes( 'DONE')) 
                    {
                        status_div.classList.remove('running');
                        status_div.classList.add('complete');
                        status_div.textContent = 'Code Approved!';
                        throw new Error('Stream complete');  // Stops retry
                    }
                    if (data.includes( 'tool-websearch')) 
                    {
                        status_div.classList.remove('complete');
                        status_div.classList.add('running');
                        status_div.textContent = 'Searching Web';

                    }
                    if (data.includes( 'tool-readurl')) 
                    {
                        status_div.textContent = 'Checking Website';

                    }
                }
                ///////////////////////
                //  Accumulate Stream
                ///////////////////////
                else if( type == 'text')
                {
                    // accumulate stream
                    if  (tag == 'llm')
                    {
                        output_store_llm.textContent += data ;
                        output_display_llm.scrollTop = output_display_llm.scrollHeight;
                    }
                    else if  (tag == 'code')
                    {
                        const code = getMonacoContent(cellId);
                        setMonacoContent(cellId,code+data);
                    }

                }
                else if(type == 'output')
                {
                        // Unit doesnt stream atm so we replace
                        output_store_code.textContent = JSON.stringify(data);
                        requestAnimationFrame(() => {
                            output_display_code.scrollTop = output_display_code.scrollHeight;
                        });
                }

            }
            // Print error 
            catch(error){
                if (error.message === 'Stream complete') throw error; 

                if  (tag == 'llm')
                {
                    output_store_llm.textContent += error_text ;
                }
                else if  (tag == 'code')
                {
                    const code = getMonacoContent(cellId);
                    setMonacoContent(cellId,code+error_text);
                }
                else if  (tag == 'unit')
                {
                    // Unit doesnt stream atm so we replace
                    output_store_code.textContent = JSON.stringify([{
                        output_type: 'error',
                        ename: 'StreamError',
                        evalue: 'Error in stream',
                        traceback: [error_text]
                    }]);
                }
            }
        },
        onerror(error) {

            throw error;  // Throwing from onerror stops retry

        },
    });
    })();
}
"""

CODE_CELL_SCRIPT = CELL_SCRIPT + AGENT_SCRIPT


In [ ]:
CSS = """
.markdown-body { 
    background-color: bg-gray-900 !important; 
}
.markdown-body ul { 
    list-style-type: disc !important; 
    padding-left: 2em !important; 
}
.markdown-body ol { 
    list-style-type: decimal !important; 
    padding-left: 2em !important; 
}
[data-cell-id].minimized .output-display-llm,
[data-cell-id].minimized .output-display-code {
    display: none !important;
}
[data-cell-id].minimized .content-edit,
[data-cell-id].minimized .content-render,
[data-cell-id].minimized .output-display,
[data-cell-id].minimized .text-gray-400 {
    display: none;
}
[data-cell-id].minimized .content-edit,
[data-cell-id].minimized .content-render {
    display: none !important;
}
[data-cell-id].minimized .monaco-editor,
[data-cell-id].minimized .content-edit,
[data-cell-id].minimized .content-render,
[data-cell-id].minimized .output-display {
    max-height: 0 !important;
    overflow: hidden;
}
[data-cell-id].minimized .bg-gray-800 {
    opacity: 0.5 !important;
}
body {
    background-color: #070707 !important;  /* 95% black = 5% brightness */
}
.markdown-body { 
    background-color: bg-gray-900 !important;
    font-size: 0.825rem !important;  /* ← Smaller base size */
}
.markdown-body h1 { font-size: 1.25rem !important; }
.markdown-body h2 { font-size: 1.1rem !important; }
.markdown-body h3 { font-size: 1.0rem !important; }
.markdown-body ul, .markdown-body ol { 
    list-style-type: disc !important; 
    padding-left: 2em !important;
    font-size: 0.825rem !important;  /* ← Match body */
}
.content-edit {
    font-size: 0.825rem !important;  /* ← Textarea size */
}

.content-render {
    background-color: #1a1a1a  !important;  /* ← Medium grey for rendered input */
}
.running { color: #ef4444 !important; }
.complete { color: #22c55e !important; }
#notebook-container {
    height: calc(100vh - 64px); 
    overflow-y: auto;
.toggle-edit:checked ~ .content-edit { display: block; }
.toggle-edit:checked ~ .content-render { display: none; }
.toggle-edit:not(:checked) ~ .content-edit { display: none; }
.toggle-edit:not(:checked) ~ .content-render { display: block; }
"""